In [9]:
import pandas as pd
import copy
import time

In [10]:
all_Gu = ['강남', '강동', '강북', '강서', '관악', '광진', '구로', '금천',
      '노원', '도봉', '동대문', '동작', '마포', '서대문', '서초', '성동', '성북', '송파',
      '양천', '영등포', '용산', '은평', '종로', '중', '중랑']

In [208]:
file_path1 = './인구+교통+POI+최단거리/cellid+인구+교통량+POI+최단거리_'  # 인구수, 교통량, 관심지점수, 가장 가까운 충전소까지의 거리
file_path2 = './cell중심간 최단거리/중심간 최단거리_'  # 각 cell의 중심간의 최단거리쌍
file_path3 = './결과/입지선정cell_'

exec_Gu = ['양천', '영등포', '용산', '은평']  # 실행할 자치구의 집합

weights = {'인구':0.2, '교통량':0.4, 'POI':0.4}  # [총인구, 교통량, 관심지점수]를 입지매력점수에 반영할 가중치

num_new_stations = 4  # 추가할 충전소 의 개수

In [209]:
# 각 cell의 입지매력점수를 구해서 리턴하는 함수
def get_attr_score(df):
    df_temp = copy.deepcopy(df)
    # PERCENTILE RANK값에 가중치를 곱해서 입지매력점수 구하기
    df_temp['총인구_score'] = [r / len(df_temp) * weights['인구'] for r in df_temp['총인구'].rank(method='average')]
    df_temp['교통량_score'] = [r / len(df_temp) * weights['교통량'] for r in df_temp['교통량'].rank(method='average')]
    df_temp['관심지점수_score'] = [r / len(df_temp) * weights['POI'] for r in df_temp['관심지점수'].rank(method='min')]
    
    # 만점 10.0, 각 점수를 더해서 10점 만점으로 스케일한다
    #df['attr_score'] = [i+j for i in df_temp['총인구_score'] for j in df_temp['교통량_rank']]
    df['attr_score'] = df_temp[['총인구_score', '교통량_score', '관심지점수_score']].sum(axis=1) * 10
    
    return df
    

In [210]:
# cell별로 입지 잠재력 점수를 구하고 총합을 리턴한다.
# 잠재력 점수: 거리점수와 매력점수의 가중 평균
#               거리점수 * 가중치1 + 매력점수 * 가중치2
# 거리점수: 모든 cell의 중심에서 가장 가까운 충전소(가상충전소 포함)까지의 거리의 합을
#           원래상태의 거리 합에서 나눈 값
# 매력점수: 추가되는 충전소 cell들의 매력점수의 평균
def get_sum_pot_score(df, df_sd, cur_stations):
            
    # 거리점수 구하기
    # 1번 cell부터 돌면서
    # cell의 중심에서 가장 가까운 충전소까지의 거리를 새로 산정하고
    # 처음에 구해놓은 최단거리합(SUM_SD)에서 새로 계산된 거리합을 나눈다
    df_temp = copy.deepcopy(df)
    
    if pd.__version__[0] == '0':
        list_df = df['최단거리'].tolist()  # 시간단축을 위해 데이터프레임을 리스트로 변환한다
        list_df_sd = df_sd['total_cost'].tolist()  # 시간단축을 위해 데이터프레임을 리스트로 변환한다
    else:
        list_df = df['최단거리'].to_list()  # 시간단축을 위해 데이터프레임을 리스트로 변환한다
        list_df_sd = df_sd['total_cost'].to_list()  # 시간단축을 위해 데이터프레임을 리스트로 변환한다
        
    SUM_SD_cur = 0
    for i in range(len(df)):

        shortest_dist = list_df[i]
        for j in cur_stations:
            #dist = df_sd[(df_sd['origin_id']==i+1)&(df_sd['destination_id']== j+1)].iloc[0, 5]  # 'total_cost'를 가져온다
            #dist = df_sd.iloc[i * len(df) + j, 5]
            dist = list_df_sd[i * len(df) + j]
            if dist == 0:  # dist = 0일 경우는 자신과의 거리이므로 제외한다.
                continue
                
            #print('i: {}, j:{}   dist= :{}'.format(i, j, dist))
            if dist < shortest_dist:
                shortest_dist = dist
            
        #df_temp.loc[i, '최단거리'] = shortest_dist  # 갱신된 최단거리
        SUM_SD_cur += shortest_dist
 
    #SUM_SD_cur = df_temp['최단거리'].sum()  # 현재시점에서 각 cell의 중심에서 가장 가까운 충전소까지의 거리의 총합
    
    # 거리점수
    #dist_score = (SUM_SD / SUM_SD_cur - 1) * 100 # 최단거리합이 짧을 수록 점수가 높다
    dist_score = round((1 - SUM_SD_cur / SUM_SD) * 40, 3) # 최단거리합이 짧을 수록 점수가 높다
    
    # 매력점수: 후보cell들의 매력점수의 평균
    attr_score = round(df_temp.loc[cur_stations, ['attr_score']].mean()[0], 3)
    
    total_score = dist_score + attr_score
    #print('거리점수: {:5.3f}   매력점수: {:5.3f}   잠재력점수: {:5.3f}'. format(dist_score, attr_score, total_score))
    
    return dist_score, attr_score, total_score

In [211]:
# 입지 잠재력점수가 최대가 되는 입지를 결정해서 리턴하는 함수
def get_optimum_stations(df, df_sd):
    # first_cells부터 시작해서 last_cells 에 도달할 때 까지 for 루프를 돈다
    # cell의 index는 0부터 시작한다
    first_cells = [i for i in range(num_new_stations)]  # 첫 n개의 cell
    last_cells = [len(df) -1 - i for i in reversed(range(num_new_stations))]   # 마지막 n개의 cell

    #print('first_cells = {}   last_cells = {}'.format(first_cells, last_cells))
    cur_stations = copy.deepcopy(first_cells)
    
    start = time.time()
    max_pot_score = 0
    max_pot_cells = [-1 for i in range(num_new_stations)]
    counter = 0
    
    TOP = 100
    list_Top_distance = [0] * TOP
    list_Top_stations = [[-1]*num_new_stations]*TOP
    list_attr_score =  [0] * TOP
    while True:
        
        # get_sum_pot_score()를 불러 입지 잠재력점수의 합을 구한다
        d_score, a_score, cur_pot_score = get_sum_pot_score(df, df_sd, cur_stations)
        
        if cur_pot_score > max_pot_score:
            max_pot_score = cur_pot_score
            max_pot_cells = copy.deepcopy(cur_stations)
            print('counter: {:6d}  stations: {}  잠재력점수: {:5.3f}  거리점수: {:5.3f}  매력점수: {:5.3f}'.
                  format(counter, cur_stations, cur_pot_score, d_score, a_score))
        
        # 거리점수를 받아서 Top리스트의 해당 순위에 삽입한다
        for i in range(TOP):
            if d_score > list_Top_distance[i]:
                for j in reversed(range(i+1, len(list_Top_distance))):
                    list_Top_distance[j] = list_Top_distance[j-1]
                    list_Top_stations[j] = list_Top_stations[j-1]
                    list_attr_score[j] = list_attr_score[j-1]
                
                list_Top_distance[i] = d_score
                #list_Top_stations[i] = copy.deepcopy(cur_stations)
                list_Top_stations[i] = '(' + ', '.join(str(s) for s in [k + 1 for k in cur_stations]) + ')'  # 엑셀로 저장할 때 리스트는 오류가 나므로 이렇게 조치함
                #list_Top_stations[i] = [k + 1 for k in cur_stations]
                ' '.join(str(s) for s in b)
                list_attr_score[i] = a_score
                #print(list_Top_distance)
                #print(list_Top_stations)
                #print(list_attr_score)
                break
                    
        if counter % 1000 == 0:
            print('counter: {:10d}  {}'.format(counter, cur_stations))
            
        
        # last_cells에 도달하면 빠져 나간다
        if cur_stations == last_cells:
            break

        # n번째 충전소 위치를 다음 cell로 옮긴다.
        stay_flg = False
        for i in reversed(range(num_new_stations)):
            # 마지막 cell에 도착했으면 pass
            if cur_stations[i] == last_cells[i]:
                stay_flg = True
            else:
                cur_stations[i] = cur_stations[i] + 1
                
                # 마지막 충전소를 제외한 충전소의 위치를 옮겼을 때는
                # 그 이후의 충전소 cell의 위치를 초기화한다.
                if i < num_new_stations - 1:
                    for j in range(i + 1, num_new_stations):
                        cur_stations[j] = cur_stations[j-1] + 1
                    
                    stay_flg = False
        
            if not(stay_flg):
                break
            
        counter += 1
    
    print('counter = {}   last_cur_stations: {}   Elapsed time : {}분'.format(counter, cur_stations, round((time.time() - start)/60, 2)))
    print('최고 잠재력점수: {:5.3f}   선정된 cell: {}'.format(max_pot_score, max_pot_cells))

    # 세개의 리스트를 하나의 데이터프레임으로 합친다
    df_all = pd.DataFrame({ '충전소 ID':list_Top_stations, '거리점수':list_Top_distance, '매력점수':list_attr_score}, columns = ['충전소 ID', '거리점수', '매력점수'])
    df_all['총점수'] = df_all[['거리점수', '매력점수']].sum(axis=1)
    #print(df_all)
    return df_all

In [212]:
# 데이터프레임을 받아 새 시트에 입력하는 함수
def to_excel(df, gu):
    sheet_name = gu
    wb = openpyxl.load_workbook(excel_path)
    wb.create_sheet(title = sheet_name)
    ws = wb[sheet_name]

    # 데이터프레임을 'Keyword News'시트에 row단위로 입력
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

    # 저장
    wb.save(excel_path)
    wb.close()

In [ ]:
import openpyxl
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows

# 오늘 날짜의 새로운 엑셀 파일을 생성
today = datetime.today()
excel_path = './excel/{}{}{}_{}{}.xlsx'.format(today.year, today.month, today.day, today.hour, today.minute)

# 오늘 날짜의 엑셀파일 생성하고 저장
wb = openpyxl.Workbook()
wb.save(excel_path)
#wb.remove_sheet(wb['Sheet'])
wb.close()

for gu in exec_Gu:
    df_all = pd.read_csv(file_path1 + gu + '구.csv', engine='python', encoding='CP949')
    df_sd_cell = pd.read_csv(file_path2 + gu + '구.csv', engine='python', encoding='CP949')
    #df_all = df_all.fillna(0)
    
    print('*****'*10)
    print('*****', gu )
    print('*****'*10)
    SUM_SD = df_all['최단거리'].sum()  # 현재시점에서 각 cell의 중심에서 가장 가까운 충전소까지의 거리의 총합
    
    # 입지매력점수 구하기(10점 만점)
    df_all = get_attr_score(df_all)
    
    # 입지 잠재력점수가 최대가 되는 입지 선정(max_station의 수만큼 cell을 선택)
    df_all = get_optimum_stations(df_all, df_sd_cell)
    
    # 엑셀 파일을 열어 해당 자치구의 시트를 생성하고 데이터프레임을 저장함
    to_excel(df_all, gu)
    
print('입지분석완료')

**************************************************
***** 양천
**************************************************
counter:      0  stations: [0, 1, 2, 3]  잠재력점수: 3.125  거리점수: 3.028  매력점수: 0.097
counter:          0  [0, 1, 2, 3]
counter:      2  stations: [0, 1, 2, 5]  잠재력점수: 4.029  거리점수: 3.040  매력점수: 0.989
counter:      4  stations: [0, 1, 2, 7]  잠재력점수: 4.315  거리점수: 3.923  매력점수: 0.392
counter:     12  stations: [0, 1, 2, 15]  잠재력점수: 4.475  거리점수: 4.089  매력점수: 0.386
counter:     14  stations: [0, 1, 2, 17]  잠재력점수: 5.444  거리점수: 4.308  매력점수: 1.136
counter:     16  stations: [0, 1, 2, 19]  잠재력점수: 5.773  거리점수: 4.046  매력점수: 1.727
counter:     23  stations: [0, 1, 2, 26]  잠재력점수: 6.149  거리점수: 4.194  매력점수: 1.955
counter:    107  stations: [0, 1, 3, 26]  잠재력점수: 6.212  거리점수: 4.280  매력점수: 1.932
counter:    190  stations: [0, 1, 4, 26]  잠재력점수: 6.234  거리점수: 4.194  매력점수: 2.040
counter:    263  stations: [0, 1, 5, 17]  잠재력점수: 6.455  거리점수: 4.449  매력점수: 2.006
counter:    265  stations: [0, 1, 5, 19]  잠재력점수:

counter:     170000  [1, 25, 55, 69]
counter:     171000  [1, 26, 35, 69]
counter:     172000  [1, 26, 61, 68]
counter:     173000  [1, 27, 40, 54]
counter:     174000  [1, 27, 72, 78]
counter:     175000  [1, 28, 46, 82]
counter:     176000  [1, 29, 33, 40]
counter:     177000  [1, 29, 56, 74]
counter:     178000  [1, 30, 41, 48]
counter:     179000  [1, 30, 75, 79]
counter:     180000  [1, 31, 52, 68]
counter:     181000  [1, 32, 40, 69]
counter:     182000  [1, 32, 73, 79]
counter:     183000  [1, 33, 54, 85]
counter:     184000  [1, 34, 44, 82]
counter:     185000  [1, 35, 37, 78]
counter:     186000  [1, 35, 65, 84]
counter:     187000  [1, 36, 55, 74]
counter:     188000  [1, 37, 49, 56]
counter:     189000  [1, 38, 44, 80]
counter:     190000  [1, 39, 41, 84]
counter:     191000  [1, 39, 78, 85]
counter:     192000  [1, 40, 72, 73]
counter:     193000  [1, 41, 69, 86]
counter:     194000  [1, 42, 70, 79]
counter:     195000  [1, 43, 74, 75]
counter:     196000  [1, 45, 46, 53]
c

counter:     393000  [3, 48, 65, 69]
counter:     394000  [3, 50, 52, 64]
counter:     395000  [3, 51, 65, 70]
counter:     396000  [3, 53, 59, 61]
counter:     397000  [3, 55, 58, 65]
counter:     398000  [3, 57, 61, 84]
counter:     399000  [3, 59, 72, 80]
counter:     400000  [3, 62, 74, 77]
counter:     401000  [3, 66, 76, 84]
counter:     402000  [3, 73, 80, 84]
counter:     403000  [4, 5, 13, 79]
counter:     404000  [4, 5, 29, 31]
counter:     405000  [4, 5, 49, 81]
counter:     406000  [4, 6, 10, 64]
counter:     407000  [4, 6, 25, 29]
counter:     408000  [4, 6, 43, 84]
counter:     409000  [4, 7, 8, 59]
counter:     410000  [4, 7, 22, 58]
counter:     411000  [4, 7, 40, 59]
counter:     412000  [4, 7, 72, 83]
counter:     413000  [4, 8, 21, 42]
counter:     414000  [4, 8, 38, 73]
counter:     415000  [4, 8, 67, 87]
counter:     416000  [4, 9, 21, 39]
counter:     417000  [4, 9, 38, 70]
counter:     418000  [4, 9, 67, 84]
counter:     419000  [4, 10, 22, 48]
counter:     42000

counter:     614000  [6, 18, 28, 43]
counter:     615000  [6, 18, 48, 73]
counter:     616000  [6, 19, 23, 70]
counter:     617000  [6, 19, 42, 44]
counter:     618000  [6, 19, 79, 82]
counter:     619000  [6, 20, 37, 68]
counter:     620000  [6, 20, 65, 74]
counter:     621000  [6, 21, 34, 76]
counter:     622000  [6, 21, 59, 76]
counter:     623000  [6, 22, 33, 49]
counter:     624000  [6, 22, 56, 83]
counter:     625000  [6, 23, 32, 87]
counter:     626000  [6, 23, 56, 67]
counter:     627000  [6, 24, 33, 80]
counter:     628000  [6, 24, 57, 84]
counter:     629000  [6, 25, 35, 84]
counter:     630000  [6, 25, 61, 83]
counter:     631000  [6, 26, 39, 56]
counter:     632000  [6, 26, 69, 81]
counter:     633000  [6, 27, 44, 61]
counter:     634000  [6, 28, 30, 43]
counter:     635000  [6, 28, 51, 77]
counter:     636000  [6, 29, 36, 69]
counter:     637000  [6, 29, 63, 70]
counter:     638000  [6, 30, 45, 59]
counter:     639000  [6, 31, 34, 36]
counter:     640000  [6, 31, 58, 64]
c

counter:     827000  [8, 62, 74, 77]
counter:     828000  [8, 66, 76, 84]
counter:     829000  [8, 73, 80, 84]
counter:     830000  [9, 10, 19, 51]
counter:     831000  [9, 10, 36, 48]
counter:     832000  [9, 10, 62, 73]
counter:     833000  [9, 11, 21, 68]
counter:     834000  [9, 11, 39, 51]
counter:     835000  [9, 11, 69, 76]
counter:     836000  [9, 12, 25, 39]
counter:     837000  [9, 12, 44, 51]
counter:     838000  [9, 13, 14, 75]
counter:     839000  [9, 13, 30, 43]
counter:     840000  [9, 13, 51, 77]
counter:     841000  [9, 14, 20, 30]
counter:     842000  [9, 14, 37, 44]
counter:     843000  [9, 14, 64, 72]
counter:     844000  [9, 15, 27, 33]
counter:     845000  [9, 15, 46, 83]
counter:     846000  [9, 16, 19, 56]
counter:     847000  [9, 16, 36, 53]
counter:     848000  [9, 16, 62, 78]
counter:     849000  [9, 17, 28, 74]
counter:     850000  [9, 17, 49, 66]
counter:     851000  [9, 18, 23, 33]
counter:     852000  [9, 18, 41, 52]
counter:     853000  [9, 18, 75, 83]
c

counter:    1045000  [11, 58, 64, 80]
counter:    1046000  [11, 61, 62, 73]
counter:    1047000  [11, 64, 70, 80]
counter:    1048000  [11, 69, 73, 80]
counter:    1049000  [12, 13, 15, 28]
counter:    1050000  [12, 13, 30, 68]
counter:    1051000  [12, 13, 52, 67]
counter:    1052000  [12, 14, 20, 55]
counter:    1053000  [12, 14, 37, 69]
counter:    1054000  [12, 14, 65, 75]
counter:    1055000  [12, 15, 27, 58]
counter:    1056000  [12, 15, 47, 68]
counter:    1057000  [12, 16, 19, 81]
counter:    1058000  [12, 16, 36, 78]
counter:    1059000  [12, 16, 63, 79]
counter:    1060000  [12, 17, 29, 41]
counter:    1061000  [12, 17, 50, 54]
counter:    1062000  [12, 18, 23, 58]
counter:    1063000  [12, 18, 41, 77]
counter:    1064000  [12, 18, 77, 87]
counter:    1065000  [12, 19, 36, 39]
counter:    1066000  [12, 19, 62, 64]
counter:    1067000  [12, 20, 32, 38]
counter:    1068000  [12, 20, 54, 81]
counter:    1069000  [12, 21, 29, 61]
counter:    1070000  [12, 21, 50, 74]
counter:    

In [385]:
aa = df_sd_cell[(df_sd_cell['origin_id']==1)&(df_sd_cell['destination_id']==3)]